### Model Training 

#### Import libraries

In [16]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

# modeling
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

import warnings 
warnings.filterwarnings('ignore')

#### Import dataset

In [6]:
df = pd.read_csv('data/stud.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


#### Train data and test data

In [7]:
X = df.drop(columns=['math_score'], axis=1)
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [8]:
y=df['math_score']
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

#### Data transformation

In [12]:
# numerical variables list
num_variables = X.select_dtypes(exclude='object').columns

# categorical variables list
cat_variables = X.select_dtypes(include='object').columns

# instantiating the scaler and OHE 
num_transformer = StandardScaler()
ohe_transformer = OneHotEncoder()

# applying the columns transformations
preprocessor = ColumnTransformer(
[
    ('OneHotEncoder', ohe_transformer, cat_variables),
    ('StandardScaler', num_transformer, num_variables)
]
)

In [13]:
X = preprocessor.fit_transform(X)
X.shape

(1000, 19)

#### Split data in train and test

In [17]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(800, 19)
(200, 19)
(800,)
(200,)


#### Metrics evaluation function

In [24]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r_two = r2_score(true, predicted)
    return mae, rmse, r_two

In [25]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso,
    "Ridge": Ridge,
    "K-Neighbors Regressor" : KNeighborsRegressor,
    "Decision Tree":DecisionTreeRegressor,
    "Random Forest":RandomForestRegressor,
    "XGBRegressor":XGBRegressor,
    "CatBoost Regressor":CatBoostRegressor,
    "AdaBoost Regressor":AdaBoostRegressor
}

model_list = []

r2_list = []

for i in range(len(list(models))):
    model  = list(models.values())[i]
    model.fit(X_train, y_train)
    
    # make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # evaluate train and test dataset
    model_train_mae, model_train_rmse, model_train_r2=evaluate_model(y_train,y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2=evaluate_model(y_test, y_test_pred)
    
    # printing model performance
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.3250
- Mean Absolute Error: 4.2674
- R2 Score: 0.8742
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.3898
- Mean Absolute Error: 4.2071
- R2 Score: 0.8806




TypeError: fit() missing 1 required positional argument: 'y'